In [128]:
import numpy as np
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [214]:
# Input examples of various LPs
X_np = np.array([[1, 1, 2, 1, 2, 3, 4, -1, -1, 0, -1],
             [1, 2, 1, 2, 1, 3, 4, -1, 0, -1, -1],
             [2, 1, 1, 1, 1, 1, 4, -1, -1, 0, -1],
             [1, 1, 2, 2, 3, 1, 4, -1, -1, 0, -1],
             [1, 1, 2, 2, 1, 3, 4, -1, -1, 0, -1],
             [1, 1, 2, 1, 3, 2, 4, -1, -1, 0, -1],
             [1, 1, 2, 3, 2, 1, 4, -1, -1, 0, -1],
             [1, 1, 2, 3, 1, 2, 4, -1, -1, 0, -1],
             [1, 1, 2, 1, 2, 3, 4, 0, -1, -1, -1],
             [1, 1, 2, 1, 2, 3, 4, -1, 0, -1, -1],
             [1, 1, 2, 2, 3, 1, 4, -1, 0, -1, -1],
             [1, 1, 2, 2, 3, 1, 4, 0, -1, -1, -1],
             [1, 1, 2, 2, 1, 3, 4, -1, 0, -1, -1],
             [1, 1, 2, 2, 1, 3, 4, 0, -1, -1, -1],
             [1, 1, 2, 1, 3, 2, 4, -1, 0, -1, -1],
             [1, 1, 2, 1, 3, 2, 4, 0, -1, -1, -1],
             [1, 1, 2, 3, 2, 1, 4, -1, 0, -1, -1],
             [1, 1, 2, 3, 2, 1, 4, 0, -1, -1, -1],
             [1, 1, 2, 3, 1, 2, 4, -1, 0, -1, -1],
             [1, 1, 2, 3, 1, 2, 4, 0, -1, -1, -1],
             [2, 1, 1, 1, 1, 1, 4, -1, 0, -1, -1],
             [2, 1, 1, 1, 1, 1, 4, 0, -1, -1, -1],
             [2, 1, 1, 1, 1, 1, 4, -1, -1, -1, -1],
             [2, 1, 1, 2, 1, 1, 4, -1, 0, -1, -1],
             [2, 1, 1, 2, 1, 1, 4, 0, -1, -1, -1],
             [3, 1, 1, 2, 1, 1, 4, -1, -1, 0, -1],
             [3, 1, 1, 2, 1, 1, 4, 0, -1, -1, -1],
             [3, 1, 1, 2, 1, 1, 4, -1, 0, -1, -1],
             [3, 1, 1, 1, 2, 1, 4, -1, -1, 0, -1],
             [3, 1, 1, 1, 2, 1, 4, 0, -1, -1, -1],
             [3, 1, 1, 1, 2, 1, 4, -1, 0, -1, -1],
             [3, 1, 1, 1, 1, 2, 4, -1, -1, 0, -1],
             [3, 1, 1, 1, 1, 2, 4, 0, -1, -1, -1],
             [3, 1, 1, 1, 1, 2, 4, -1, 0, -1, -1],
             [1, 3, 1, 2, 1, 1, 4, -1, -1, 0, -1],
             [1, 3, 1, 2, 1, 1, 4, 0, -1, -1, -1],
             [1, 3, 1, 2, 1, 1, 4, -1, 0, -1, -1],
             [1, 3, 1, 1, 2, 1, 4, -1, -1, 0, -1],
             [1, 3, 1, 1, 2, 1, 4, 0, -1, -1, -1],
             [1, 3, 1, 1, 2, 1, 4, -1, 0, -1, -1],
             [1, 3, 1, 1, 1, 2, 4, -1, -1, 0, -1],
             [1, 3, 1, 1, 1, 2, 4, 0, -1, -1, -1],
             [1, 3, 1, 1, 1, 2, 4, -1, 0, -1, -1],
             [1, 1, 3, 2, 1, 1, 4, -1, -1, 0, -1],
             [1, 1, 3, 2, 1, 1, 4, 0, -1, -1, -1],
             [1, 1, 3, 2, 1, 1, 4, -1, 0, -1, -1],
             [1, 1, 3, 1, 2, 1, 4, -1, -1, 0, -1],
             [1, 1, 3, 1, 2, 1, 4, 0, -1, -1, -1],
             [1, 1, 3, 1, 2, 1, 4, -1, 0, -1, -1],
             [1, 1, 3, 1, 1, 2, 4, -1, -1, 0, -1],
             [1, 1, 3, 1, 1, 2, 4, 0, -1, -1, -1],
             [1, 1, 3, 1, 1, 2, 4, -1, 0, -1, -1],
             [2, 3, 1, 2, 1, 1, 3, -1, -1, 0, -1],
             [2, 3, 1, 2, 1, 1, 3, 0, -1, -1, -1],
             [2, 3, 1, 2, 1, 1, 3, -1, 0, -1, -1],
             [2, 3, 1, 1, 2, 1, 3, -1, -1, 0, -1],
             [2, 3, 1, 1, 2, 1, 3, 0, -1, -1, -1],
             [2, 3, 1, 1, 2, 1, 3, -1, 0, -1, -1],
             [2, 3, 1, 1, 1, 2, 3, -1, -1, 0, -1],
             [2, 3, 1, 1, 1, 2, 3, 0, -1, -1, -1],
             [2, 3, 1, 1, 1, 2, 3, -1, 0, -1, -1],
             [2, 1, 3, 2, 1, 1, 3, -1, -1, 0, -1],
             [2, 1, 3, 2, 1, 1, 3, 0, -1, -1, -1],
             [2, 1, 3, 2, 1, 1, 3, -1, 0, -1, -1],
             [2, 1, 3, 1, 2, 1, 3, -1, -1, 0, -1],
             [2, 1, 3, 1, 2, 1, 3, 0, -1, -1, -1],
             [2, 1, 3, 1, 2, 1, 3, -1, 0, -1, -1],
             [2, 1, 3, 1, 1, 2, 3, -1, -1, 0, -1],
             [2, 1, 3, 1, 1, 2, 3, 0, -1, -1, -1],
             [2, 1, 3, 1, 1, 2, 3, -1, 0, -1, -1],
             [1, 2, 3, 2, 1, 1, 3, -1, -1, 0, -1],
             [1, 2, 3, 2, 1, 1, 3, 0, -1, -1, -1],
             [1, 2, 3, 2, 1, 1, 3, -1, 0, -1, -1],
             [1, 2, 3, 1, 2, 1, 3, -1, -1, 0, -1],
             [1, 2, 3, 1, 2, 1, 3, 0, -1, -1, -1],
             [1, 2, 3, 1, 2, 1, 3, -1, 0, -1, -1],
             [1, 2, 3, 1, 1, 2, 3, -1, -1, 0, -1],
             [1, 2, 3, 1, 1, 2, 3, 0, -1, -1, -1],
             [1, 2, 3, 1, 1, 2, 3, -1, 0, -1, -1],
             [1, 3, 2, 2, 1, 1, 3, -1, -1, 0, -1],
             [1, 3, 2, 2, 1, 1, 3, 0, -1, -1, -1],
             [1, 3, 2, 2, 1, 1, 3, -1, 0, -1, -1],
             [1, 3, 2, 1, 2, 1, 3, -1, -1, 0, -1],
             [1, 3, 2, 1, 2, 1, 3, 0, -1, -1, -1],
             [1, 3, 2, 1, 2, 1, 3, -1, 0, -1, -1],
             [1, 3, 2, 1, 1, 2, 3, -1, -1, 0, -1],
             [1, 3, 2, 1, 1, 2, 3, 0, -1, -1, -1],
             [1, 3, 2, 1, 1, 2, 3, -1, 0, -1, -1],
             [3, 1, 2, 2, 1, 1, 3, -1, -1, 0, -1],
             [3, 1, 2, 2, 1, 1, 3, 0, -1, -1, -1],
             [3, 1, 2, 2, 1, 1, 3, -1, 0, -1, -1],
             [3, 1, 2, 1, 2, 1, 3, -1, -1, 0, -1],
             [3, 1, 2, 1, 2, 1, 3, 0, -1, -1, -1],
             [3, 1, 2, 1, 2, 1, 3, -1, 0, -1, -1],
             [3, 1, 2, 1, 1, 2, 3, -1, -1, 0, -1],
             [3, 1, 2, 1, 1, 2, 3, 0, -1, -1, -1],
             [3, 1, 2, 1, 1, 2, 3, -1, 0, -1, -1],
             [3, 2, 1, 2, 1, 1, 3, -1, -1, 0, -1],
             [3, 2, 1, 2, 1, 1, 3, 0, -1, -1, -1],
             [3, 2, 1, 2, 1, 1, 3, -1, 0, -1, -1],
             [3, 2, 1, 1, 2, 1, 3, -1, -1, 0, -1],
             [3, 2, 1, 1, 2, 1, 3, 0, -1, -1, -1],
             [3, 2, 1, 1, 2, 1, 3, -1, 0, -1, -1],
             [3, 2, 1, 1, 1, 2, 3, -1, -1, 0, -1],
             [3, 2, 1, 1, 1, 2, 3, 0, -1, -1, -1],
             [3, 2, 1, 1, 1, 2, 3, -1, 0, -1, -1],
             [1, 2, 3, 2, 3, 1, 4, -1, -1, 0, -1],
             [1, 2, 3, 2, 3, 1, 4, 0, -1, -1, -1],
             [1, 2, 3, 2, 3, 1, 4, -1, 0, -1, -1],
             [1, 2, 3, 2, 1, 3, 4, -1, -1, 0, -1],
             [1, 2, 3, 2, 1, 3, 4, 0, -1, -1, -1],
             [1, 2, 3, 2, 1, 3, 4, -1, 0, -1, -1],
             [1, 2, 3, 1, 2, 3, 4, -1, -1, 0, -1],
             [1, 2, 3, 1, 2, 3, 4, 0, -1, -1, -1],
             [1, 2, 3, 1, 2, 3, 4, -1, 0, -1, -1],
             [1, 2, 3, 1, 3, 2, 4, -1, -1, 0, -1],
             [1, 2, 3, 1, 3, 2, 4, 0, -1, -1, -1],
             [1, 2, 3, 1, 3, 2, 4, -1, 0, -1, -1],
             [1, 3, 2, 2, 3, 1, 4, -1, -1, 0, -1],
             [1, 3, 2, 2, 3, 1, 4, 0, -1, -1, -1],
             [1, 3, 2, 2, 3, 1, 4, -1, 0, -1, -1],
             [1, 3, 2, 2, 1, 3, 4, -1, -1, 0, -1],
             [1, 3, 2, 2, 1, 3, 4, 0, -1, -1, -1],
             [1, 3, 2, 2, 1, 3, 4, -1, 0, -1, -1],
             [1, 3, 2, 1, 3, 2, 4, -1, -1, 0, -1],
             [1, 3, 2, 1, 3, 2, 4, 0, -1, -1, -1],
             [1, 3, 2, 1, 3, 2, 4, -1, 0, -1, -1],
             [1, 3, 2, 1, 2, 3, 4, -1, -1, 0, -1],
             [1, 3, 2, 1, 2, 3, 4, 0, -1, -1, -1],
             [1, 3, 2, 1, 2, 3, 4, -1, 0, -1, -1],
             [2, 1, 3, 2, 3, 1, 4, -1, -1, 0, -1],
             [2, 1, 3, 2, 3, 1, 4, 0, -1, -1, -1],
             [2, 1, 3, 2, 3, 1, 4, -1, 0, -1, -1],
             [2, 1, 3, 2, 1, 3, 4, -1, -1, 0, -1],
             [2, 1, 3, 2, 1, 3, 4, 0, -1, -1, -1],
             [2, 1, 3, 2, 1, 3, 4, -1, 0, -1, -1],
             [2, 1, 3, 1, 3, 2, 4, -1, -1, 0, -1],
             [2, 1, 3, 1, 3, 2, 4, 0, -1, -1, -1],
             [2, 1, 3, 1, 3, 2, 4, -1, 0, -1, -1],
             [2, 1, 3, 1, 2, 3, 4, -1, -1, 0, -1],
             [2, 1, 3, 1, 2, 3, 4, 0, -1, -1, -1],
             [2, 1, 3, 1, 2, 3, 4, -1, 0, -1, -1],
             [1, 3, 2, 3, 1, 2, 4, -1, -1, 0, -1],
             [1, 3, 2, 3, 1, 2, 4, 0, -1, -1, -1],
             [1, 3, 2, 3, 1, 2, 4, -1, 0, -1, -1],
             [1, 3, 2, 3, 2, 1, 4, -1, -1, 0, -1],
             [1, 3, 2, 3, 2, 1, 4, 0, -1, -1, -1],
             [1, 3, 2, 3, 2, 1, 4, -1, 0, -1, -1],
             [3, 2, 1, 3, 1, 2, 4, -1, -1, 0, -1],
             [3, 2, 1, 3, 1, 2, 4, 0, -1, -1, -1],
             [3, 2, 1, 3, 1, 2, 4, -1, 0, -1, -1],
             [3, 2, 1, 3, 2, 1, 4, -1, -1, 0, -1],
             [3, 2, 1, 3, 2, 1, 4, 0, -1, -1, -1],
             [3, 2, 1, 3, 2, 1, 4, -1, 0, -1, -1],
             [1, 2, 3, 3, 1, 2, 4, -1, -1, 0, -1],
             [1, 2, 3, 3, 1, 2, 4, 0, -1, -1, -1],
             [1, 2, 3, 3, 1, 2, 4, -1, 0, -1, -1],
             [1, 2, 3, 3, 2, 1, 4, -1, -1, 0, -1],
             [1, 2, 3, 3, 2, 1, 4, 0, -1, -1, -1],
             [1, 2, 3, 3, 2, 1, 4, -1, 0, -1, -1],
             [3, 1, 2, 2, 3, 1, 4, -1, -1, 0, -1],
             [3, 1, 2, 2, 3, 1, 4, 0, -1, -1, -1],
             [3, 1, 2, 2, 3, 1, 4, -1, 0, -1, -1],
             [3, 1, 2, 2, 1, 3, 4, -1, -1, 0, -1],
             [3, 1, 2, 2, 1, 3, 4, 0, -1, -1, -1],
             [3, 1, 2, 2, 1, 3, 4, -1, 0, -1, -1],
             [3, 1, 2, 1, 3, 2, 4, -1, -1, 0, -1],
             [3, 1, 2, 1, 3, 2, 4, 0, -1, -1, -1],
             [3, 1, 2, 1, 3, 2, 4, -1, 0, -1, -1],
             [3, 1, 2, 1, 2, 3, 4, -1, -1, 0, -1],
             [3, 1, 2, 1, 2, 3, 4, 0, -1, -1, -1],
             [3, 1, 2, 1, 2, 3, 4, -1, 0, -1, -1],
             [3, 1, 2, 3, 1, 2, 4, -1, -1, 0, -1],
             [3, 1, 2, 3, 1, 2, 4, 0, -1, -1, -1],
             [3, 1, 2, 3, 1, 2, 4, -1, 0, -1, -1],
             [3, 1, 2, 3, 2, 1, 4, -1, -1, 0, -1],
             [3, 1, 2, 3, 2, 1, 4, 0, -1, -1, -1],
             [3, 1, 2, 3, 2, 1, 4, -1, 0, -1, -1],
             [2, 3, 1, 2, 3, 1, 4, -1, -1, 0, -1],
             [2, 3, 1, 2, 3, 1, 4, 0, -1, -1, -1],
             [2, 3, 1, 2, 3, 1, 4, -1, 0, -1, -1],
             [2, 3, 1, 2, 1, 3, 4, -1, -1, 0, -1],
             [2, 3, 1, 2, 1, 3, 4, 0, -1, -1, -1],
             [2, 3, 1, 2, 1, 3, 4, -1, 0, -1, -1],
             [2, 3, 1, 1, 3, 2, 4, -1, -1, 0, -1],
             [2, 3, 1, 1, 3, 2, 4, 0, -1, -1, -1],
             [2, 3, 1, 1, 3, 2, 4, -1, 0, -1, -1],
             [2, 3, 1, 1, 2, 3, 4, -1, -1, 0, -1],
             [2, 3, 1, 1, 2, 3, 4, 0, -1, -1, -1],
             [2, 3, 1, 1, 2, 3, 4, -1, 0, -1, -1],
             [2, 3, 1, 3, 1, 2, 4, -1, -1, 0, -1],
             [2, 3, 1, 3, 1, 2, 4, 0, -1, -1, -1],
             [2, 3, 1, 3, 1, 2, 4, -1, 0, -1, -1],
             [2, 3, 1, 3, 2, 1, 4, -1, -1, 0, -1],
             [2, 3, 1, 3, 2, 1, 4, 0, -1, -1, -1],
             [2, 3, 1, 3, 2, 1, 4, -1, 0, -1, -1],
             [2, 1, 3, 3, 1, 2, 4, -1, -1, 0, -1],
             [2, 1, 3, 3, 1, 2, 4, 0, -1, -1, -1],
             [2, 1, 3, 3, 1, 2, 3, -1, 0, -1, -1],
             [2, 1, 3, 3, 2, 1, 4, -1, -1, 0, -1],
             [2, 1, 3, 3, 2, 1, 4, 0, -1, -1, -1],
             [2, 1, 3, 3, 2, 1, 4, -1, 0, -1, -1],
             [3, 2, 1, 2, 3, 1, 4, -1, -1, 0, -1],
             [3, 2, 1, 2, 3, 1, 4, 0, -1, -1, -1],
             [3, 2, 1, 2, 3, 1, 4, -1, 0, -1, -1],
             [3, 2, 1, 2, 1, 3, 4, -1, -1, 0, -1],
             [3, 2, 1, 2, 1, 3, 4, 0, -1, -1, -1],
             [3, 2, 1, 2, 1, 3, 4, -1, 0, -1, -1],
             [3, 2, 1, 1, 3, 2, 4, -1, -1, 0, -1],
             [3, 2, 1, 1, 3, 2, 4, 0, -1, -1, -1],
             [3, 2, 1, 1, 3, 2, 4, -1, 0, -1, -1],
             [3, 2, 1, 1, 2, 3, 4, -1, -1, 0, -1],
             [3, 2, 1, 1, 2, 3, 4, 0, -1, -1, -1],
             [3, 2, 1, 1, 2, 3, 4, -1, 0, -1, -1],
             [2, 3, 4, 2, 3, 4, 7, -1, -1, 0, -1],
             [2, 3, 4, 2, 3, 4, 7, 0, -1, -1, -1],
             [2, 3, 4, 2, 3, 4, 7, -1, 0, -1, -1],
             [2, 3, 4, 2, 4, 3, 7, -1, -1, 0, -1],
             [2, 3, 4, 2, 4, 3, 7, 0, -1, -1, -1],
             [2, 3, 4, 2, 4, 3, 7, -1, 0, -1, -1],
             [2, 3, 4, 2, 3, 4, 7, -1, -1, -1, -1],
             [2, 3, 4, 2, 4, 3, 7, -1, -1, -1, -1],
             [2, 3, 4, 3, 2, 4, 7, -1, -1, 0, -1],
             [2, 3, 4, 3, 2, 4, 7, 0, -1, -1, -1],
             [2, 3, 4, 3, 2, 4, 7, -1, 0, -1, -1],
             [2, 3, 4, 3, 4, 2, 7, -1, -1, 0, -1],
             [2, 3, 4, 3, 4, 2, 7, 0, -1, -1, -1],
             [2, 3, 4, 3, 4, 2, 7, -1, 0, -1, -1],
             [2, 3, 4, 3, 4, 2, 7, -1, -1, -1, -1],
             [2, 3, 4, 3, 2, 4, 7, -1, -1, -1, -1],
             [2, 3, 4, 4, 2, 3, 7, -1, -1, 0, -1],
             [2, 3, 4, 4, 2, 3, 7, 0, -1, -1, -1],
             [2, 3, 4, 4, 2, 3, 7, -1, 0, -1, -1],
             [2, 3, 4, 4, 2, 3, 7, -1, -1, -1, -1],
             [2, 3, 4, 4, 3, 2, 7, -1, -1, 0, -1],
             [2, 3, 4, 4, 3, 2, 7, 0, -1, -1, -1],
             [2, 3, 4, 4, 3, 2, 7, -1, 0, -1, -1],
             [2, 3, 4, 4, 3, 2, 7, -1, -1, -1, -1],
             [2, 4, 3, 2, 3, 4, 7, -1, -1, 0, -1],
             [2, 4, 3, 2, 3, 4, 7, 0, -1, -1, -1],
             [2, 4, 3, 2, 3, 4, 7, -1, 0, -1, -1],
             [2, 4, 3, 2, 3, 4, 7, -1, -1, -1, -1],
             [2, 4, 3, 2, 4, 3, 7, -1, -1, 0, -1],
             [2, 4, 3, 2, 4, 3, 7, 0, -1, -1, -1],
             [2, 4, 3, 2, 4, 3, 7, -1, 0, -1, -1],
             [2, 4, 3, 2, 4, 3, 7, -1, -1, -1, -1],
             [2, 4, 3, 3, 2, 4, 7, -1, -1, 0, -1],
             [2, 4, 3, 3, 2, 4, 7, 0, -1, -1, -1],
             [2, 4, 3, 3, 2, 4, 7, -1, 0, -1, -1],
             [2, 4, 3, 3, 2, 4, 7, -1, -1, -1, -1],
             [2, 4, 3, 3, 4, 2, 7, -1, -1, 0, -1],
             [2, 4, 3, 3, 4, 2, 7, 0, -1, -1, -1],
             [2, 4, 3, 3, 4, 2, 7, -1, 0, -1, -1],
             [2, 4, 3, 3, 4, 2, 7, -1, -1, -1, -1],
             [2, 4, 3, 4, 2, 3, 7, -1, -1, 0, -1],
             [2, 4, 3, 4, 2, 3, 7, 0, -1, -1, -1],
             [2, 4, 3, 4, 2, 3, 7, -1, 0, -1, -1],
             [2, 4, 3, 4, 2, 3, 7, -1, -1, -1, -1],
             [2, 4, 3, 4, 3, 2, 7, -1, -1, 0, -1],
             [2, 4, 3, 4, 3, 2, 7, 0, -1, -1, -1],
             [2, 4, 3, 4, 3, 2, 7, -1, 0, -1, -1],
             [2, 4, 3, 4, 3, 2, 7, -1, -1, -1, -1],
             [3, 2, 4, 2, 3, 4, 7, -1, -1, 0, -1],
             [3, 2, 4, 2, 3, 4, 7, 0, -1, -1, -1],
             [3, 2, 4, 2, 3, 4, 7, -1, 0, -1, -1],
             [3, 2, 4, 2, 3, 4, 7, -1, -1, -1, -1],
             [3, 2, 4, 2, 4, 3, 7, -1, -1, 0, -1],
             [3, 2, 4, 2, 4, 3, 7, 0, -1, -1, -1],
             [3, 2, 4, 2, 4, 3, 7, -1, 0, -1, -1],
             [3, 2, 4, 2, 4, 3, 7, -1, -1, -1, -1],
             [3, 2, 4, 3, 2, 4, 7, -1, -1, 0, -1],
             [3, 2, 4, 3, 2, 4, 7, 0, -1, -1, -1],
             [3, 2, 4, 3, 2, 4, 7, -1, 0, -1, -1],
             [3, 2, 4, 3, 2, 4, 7, -1, -1, -1, -1],
             [3, 2, 4, 3, 4, 2, 7, -1, -1, 0, -1],
             [3, 2, 4, 3, 4, 2, 7, 0, -1, -1, -1],
             [3, 2, 4, 3, 4, 2, 7, -1, 0, -1, -1],
             [3, 2, 4, 3, 4, 2, 7, -1, -1, -1, -1],
             [3, 2, 4, 4, 2, 3, 7, -1, -1, 0, -1],
             [3, 2, 4, 4, 2, 3, 7, 0, -1, -1, -1],
             [3, 2, 4, 4, 2, 3, 7, -1, 0, -1, -1],
             [3, 2, 4, 4, 2, 3, 7, -1, -1, -1, -1],
             [3, 2, 4, 4, 3, 2, 7, -1, -1, 0, -1],
             [3, 2, 4, 4, 3, 2, 7, 0, -1, -1, -1],
             [3, 2, 4, 4, 3, 2, 7, -1, 0, -1, -1],
             [3, 2, 4, 4, 3, 2, 7, -1, -1, -1, -1],
             [3, 4, 2, 2, 3, 4, 7, -1, -1, 0, -1],
             [3, 4, 2, 2, 3, 4, 7, 0, -1, -1, -1],
             [3, 4, 2, 2, 3, 4, 7, -1, 0, -1, -1],
             [3, 4, 2, 2, 3, 4, 7, -1, -1, -1, -1],
             [3, 4, 2, 2, 4, 3, 7, -1, -1, 0, -1],
             [3, 4, 2, 2, 4, 3, 7, 0, -1, -1, -1],
             [3, 4, 2, 2, 4, 3, 7, -1, 0, -1, -1],
             [3, 4, 2, 2, 4, 3, 7, -1, -1, -1, -1],
             [3, 4, 2, 3, 2, 4, 7, -1, -1, 0, -1],
             [3, 4, 2, 3, 2, 4, 7, 0, -1, -1, -1],
             [3, 4, 2, 3, 2, 4, 7, -1, 0, -1, -1],
             [3, 4, 2, 3, 2, 4, 7, -1, -1, -1, -1],
             [3, 4, 2, 3, 4, 2, 7, -1, -1, 0, -1],
             [3, 4, 2, 3, 4, 2, 7, 0, -1, -1, -1],
             [3, 4, 2, 3, 4, 2, 7, -1, 0, -1, -1],
             [3, 4, 2, 3, 4, 2, 7, -1, -1, -1, -1],
             [3, 4, 2, 4, 2, 3, 7, -1, -1, 0, -1],
             [3, 4, 2, 4, 2, 3, 7, 0, -1, -1, -1],
             [3, 4, 2, 4, 2, 3, 7, -1, 0, -1, -1],
             [3, 4, 2, 4, 2, 3, 7, -1, -1, -1, -1],
             [3, 4, 2, 4, 3, 2, 7, -1, -1, 0, -1],
             [3, 4, 2, 4, 3, 2, 7, 0, -1, -1, -1],
             [3, 4, 2, 4, 3, 2, 7, -1, 0, -1, -1],
             [3, 4, 2, 4, 3, 2, 7, -1, -1, -1, -1],
             [4, 2, 3, 2, 3, 4, 7, -1, -1, 0, -1],
             [4, 2, 3, 2, 3, 4, 7, 0, -1, -1, -1],
             [4, 2, 3, 2, 3, 4, 7, -1, 0, -1, -1],
             [4, 2, 3, 2, 3, 4, 7, -1, -1, -1, -1],
             [4, 2, 3, 2, 4, 3, 7, -1, -1, 0, -1],
             [4, 2, 3, 2, 4, 3, 7, 0, -1, -1, -1],
             [4, 2, 3, 2, 4, 3, 7, -1, 0, -1, -1],
             [4, 2, 3, 2, 4, 3, 7, -1, -1, -1, -1],
             [4, 2, 3, 3, 2, 4, 7, -1, -1, 0, -1],
             [4, 2, 3, 3, 2, 4, 7, 0, -1, -1, -1],
             [4, 2, 3, 3, 2, 4, 7, -1, 0, -1, -1],
             [4, 2, 3, 3, 2, 4, 7, -1, -1, -1, -1],
             [4, 2, 3, 3, 4, 2, 7, -1, -1, 0, -1],
             [4, 2, 3, 3, 4, 2, 7, 0, -1, -1, -1],
             [4, 2, 3, 3, 4, 2, 7, -1, 0, -1, -1],
             [4, 2, 3, 3, 4, 2, 7, -1, -1, -1, -1],
             [4, 2, 3, 4, 2, 3, 7, -1 ,-1, 0, -1],
             [4, 2, 3, 4, 2, 3, 7, 0, -1, -1, -1],
             [4, 2, 3, 4, 2, 3, 7, -1, 0, -1, -1],
             [4, 2, 3, 4, 2, 3, 7, -1, -1, -1, -1],
             [4, 2, 3, 4, 3, 2, 7, -1, -1, 0, -1],
             [4, 2, 3, 4, 3, 2, 7, 0, -1, -1, -1],
             [4, 2, 3, 4, 3, 2, 7, -1, 0, -1, -1],
             [4, 2, 3, 4, 3, 2, 7, -1, -1, -1, -1],
             [4, 3, 2, 2, 3, 4, 7, -1, -1 ,0, -1],
             [4, 3, 2, 2, 3, 4, 7, 0, -1, -1, -1],
             [4, 3, 2, 2, 3, 4, 7, -1, 0, -1, -1],
             [4, 3, 2, 2, 3, 4, 7, -1, -1, -1, -1],
             [4, 3, 2, 2, 4, 3, 7, -1, -1, 0, -1],
             [4, 3, 2, 2, 4, 3, 7, 0, -1, -1, -1],
             [4, 3, 2, 2, 4, 3, 7, -1, 0, -1, -1],
             [4, 3, 2, 2, 4, 3, 7, -1, -1, -1, -1],
             [4, 3, 2, 3, 2, 4, 7, -1, -1, 0, -1],
             [4, 3, 2, 3, 2, 4, 7, 0, -1, -1, -1],
             [4, 3, 2, 3, 2, 4, 7, -1, 0, -1, -1],
             [4, 3, 2, 3, 2, 4, 7, -1, -1, -1, -1],
             [4, 3, 2, 3, 4, 2, 7, -1, -1, 0, -1],
             [4, 3, 2, 3, 4, 2, 7, 0, -1, -1, -1],
             [4, 3, 2, 3, 4, 2, 7, -1, 0, -1, -1],
             [4, 3, 2, 3, 4, 2, 7, -1, -1, -1, -1],
             [4, 3, 2, 4, 2, 3, 7, -1, -1, 0, -1],
             [4, 3, 2, 4, 2, 3, 7, 0, -1, -1, -1],
             [4, 3, 2, 4, 2, 3, 7, -1, 0, -1, -1],
             [4, 3, 2, 4, 2, 3, 7, -1, -1, -1, -1],
             [4, 3, 2, 4, 3, 2, 7, -1, -1, 0, -1],
             [4, 3, 2, 4, 3, 2, 7, 0, -1, -1, -1],
             [4, 3, 2, 4, 3, 2, 7, -1, 0, -1, -1],
             [4, 3, 2, 4, 3, 2, 7, -1, -1, -1, -1],
             [3, 4, 5, 3, 4, 5, 9, -1, -1, 0, -1],
             [3, 4, 5, 3, 4, 5, 9, 0, -1, -1, -1],
             [3, 4, 5, 3, 4, 5, 9, -1, 0, -1, -1],
             [3, 4, 5, 3, 4, 5, 9, -1, -1, -1, -1],
             [3, 4, 5, 3, 5, 4, 9, -1, -1, 0, -1],
             [3, 4, 5, 3, 5, 4, 9, 0, -1, -1, -1],
             [3, 4, 5, 3, 5, 4, 9, -1, 0, -1, -1],
             [3, 4, 5, 3, 5, 4, 9, -1, -1, -1, -1],
             [3, 4, 5, 4, 3, 5, 9, -1, -1, 0, -1],
             [3, 4, 5, 4, 3, 5, 9, 0, -1, -1, -1],
             [3, 4, 5, 4, 3, 5, 9, -1, 0, -1, -1],
             [3, 4, 5, 4, 3, 5, 9, -1, -1, -1, -1],
             [3, 4, 5, 4, 5, 3, 9, -1, -1, 0, -1],
             [3, 4, 5, 4, 5, 3, 9, 0, -1, -1, -1],
             [3, 4, 5, 4, 5, 3, 9, -1 ,0, -1, -1],
             [3, 4, 5, 4, 5, 3, 9, -1, -1, -1, -1],
             [3, 4, 5, 5, 3, 4, 9, -1, -1, 0, -1],
             [3, 4, 5, 5, 3, 4, 9, 0, -1, -1, -1],
             [3, 4, 5, 5, 3, 4, 9, -1, 0, -1, -1],
             [3, 4, 5, 5, 3, 4, 9, -1, -1, -1, -1],
             [3, 4, 5, 5, 4, 3, 9, -1, -1, 0, -1],
             [3, 4, 5, 5, 4, 3, 9, 0, -1, -1, -1],
             [3, 4, 5, 5, 4, 3, 9, -1, 0, -1, -1],
             [3, 4, 5, 5, 4, 3, 9, -1, -1, -1, -1],
             [3, 5, 4, 3, 4, 5, 9, -1, -1, 0, -1],
             [3, 5, 4, 3, 4, 5, 9, 0, -1, -1, -1],
             [3, 5, 4, 3, 4, 5, 9, -1, 0, -1, -1],
             [3, 5, 4, 3, 4, 5, 9, -1, -1, -1, -1],
             [3, 5, 4, 3, 5, 4, 9, -1, -1, 0, -1],
             [3, 5, 4, 3, 5, 4, 9, 0, -1, -1, -1],
             [3, 5, 4, 3, 5, 4, 9, -1, 0, -1, -1],
             [3, 5, 4, 3, 5, 4, 9, -1, -1, -1, -1],
             [3, 5, 4, 4, 3, 5, 9, -1, -1, 0, -1],
             [3, 5, 4, 4, 3, 5, 9, 0, -1, -1, -1],
             [3, 5, 4, 4, 3, 5, 9, -1, 0, -1, -1],
             [3, 5, 4, 4, 3, 5, 9, -1, -1, -1, -1],
             [3, 5, 4, 4, 5, 3, 9, -1, -1, 0, -1],
             [3, 5, 4, 4, 5, 3, 9, 0, -1, -1, -1],
             [3, 5, 4, 4, 5, 3, 9, -1, 0, -1, -1],
             [3, 5, 4, 4, 5, 3, 9, -1, -1, -1, -1],
             [3, 5, 4, 5, 3, 4, 9, -1, -1, 0, -1],
             [3, 5, 4, 5, 3, 4, 9, 0, -1, -1, -1],
             [3, 5, 4, 5, 3, 4, 9, -1, 0, -1, -1],
             [3, 5, 4, 5, 3, 4, 9, -1, -1, -1, -1],
             [3, 5, 4, 5, 4, 3, 9, -1, -1, 0, -1],
             [3, 5, 4, 5, 4, 3, 9, 0, -1, -1, -1],
             [3, 5, 4, 5, 4, 3, 9, -1, 0, -1, -1],
             [3, 5, 4, 5, 4, 3, 9, -1, -1, -1, -1],
             [4, 3, 5, 3, 4, 5, 9, -1, -1, 0, -1],
             [4, 3, 5, 3, 4, 5, 9, 0, -1, -1, -1],
             [4, 3, 5, 3, 4, 5, 9, -1, 0, -1, -1],
             [4, 3, 5, 3, 4, 5, 9, -1, -1, -1, -1],
             [4, 3, 5, 3, 5, 4, 9, -1, -1, 0, -1],
             [4, 3, 5, 3, 5, 4, 9, 0, -1, -1, -1],
             [4, 3, 5, 3, 5, 4, 9, -1, 0, -1, -1],
             [4, 3, 5, 3, 5, 4, 9, -1, -1, -1, -1],
             [4, 3, 5, 4, 3, 5, 9, -1, -1, 0, -1],
             [4, 3, 5, 4, 3, 5, 9, 0, -1, -1, -1],
             [4, 3, 5, 4, 3, 5, 9, -1, 0, -1, -1],
             [4, 3, 5, 4, 3, 5, 9, -1, -1, -1, -1],
             [4, 3, 5, 4, 5, 3, 9, -1, -1, 0, -1],
             [4, 3, 5, 4, 5, 3, 9, 0, -1, -1, -1],
             [4, 3, 5, 4, 5, 3, 9, -1, 0, -1, -1],
             [4, 3, 5, 4, 5, 3, 9, -1, -1, -1, -1],
             [4, 3, 5, 5, 3, 4, 9, -1, -1, 0, -1],
             [4, 3, 5, 5, 3, 4, 9, 0, -1, -1, -1],
             [4, 3, 5, 5, 3, 4, 9, -1, 0, -1, -1],
             [4, 3, 5, 5, 3, 4, 9, -1, -1, -1, -1],
             [4, 3, 5, 5, 4, 3, 9, -1, -1, 0, -1],
             [4, 3, 5, 5, 4, 3, 9, 0, -1, -1, -1],
             [4, 3, 5, 5, 4, 3, 9, -1, 0, -1, -1],
             [4, 3, 5, 5, 4, 3, 9, -1, -1, -1, -1],
             [4, 5, 3, 3, 4, 5, 9, -1, -1, 0, -1],
             [4, 5, 3, 3, 4, 5, 9, 0, -1, -1, -1],
             [4, 5, 3, 3, 4, 5, 9, -1, 0, -1, -1],
             [4, 5, 3, 3, 4, 5, 9, -1, -1, -1, -1],
             [4, 5, 3, 3, 5, 4, 9, -1, -1, 0, -1],
             [4, 5, 3, 3, 5, 4, 9, 0, -1, -1, -1],
             [4, 5, 3, 3, 5, 4, 9, -1, 0, -1, -1],
             [4, 5, 3, 3, 5, 4, 9, -1, -1, -1, -1],
             [4, 5, 3, 4, 3, 5, 9, -1, -1, 0, -1],
             [4, 5, 3, 4, 3, 5, 9, 0, -1, -1, -1],
             [4, 5, 3, 4, 3, 5, 9, -1, 0, -1, -1],
             [4, 5, 3, 4, 3, 5, 9, -1, -1, -1, -1],
             [4, 5, 3, 4, 5, 3, 9, -1, -1, 0, -1],
             [4, 5, 3, 4, 5, 3, 9, 0, -1, -1, -1],
             [4, 5, 3, 4, 5, 3, 9, -1, 0, -1, -1],
             [4, 5, 3, 4, 5, 3, 9, -1, -1, -1, -1],
             [4, 5, 3, 5, 3, 4, 9, -1, -1, 0, -1],
             [4, 5, 3, 5, 3, 4, 9, 0, -1, -1, -1],
             [4, 5, 3, 5, 3, 4, 9, -1, 0, -1, -1],
             [4, 5, 3, 5, 3, 4, 9, -1, -1, -1, -1],
             [4, 5, 3, 5, 4, 3, 9, -1, -1, 0, -1],
             [4, 5, 3, 5, 4, 3, 9, 0, -1, -1, -1],
             [4, 5, 3, 5, 4, 3, 9, -1, 0, -1, -1],
             [4, 5, 3, 5, 4, 3, 9, -1, -1, -1, -1],
             [5, 3, 4, 3, 4, 5, 9, -1, -1, 0, -1],
             [5, 3, 4, 3, 4, 5, 9, 0, -1, -1, -1],
             [5, 3, 4, 3, 4, 5, 9, -1, 0, -1, -1],
             [5, 3, 4, 3, 4, 5, 9, -1, -1, -1, -1],
             [5, 3, 4, 3, 5, 4, 9, -1, -1, 0, -1],
             [5, 3, 4, 3, 5, 4, 9, 0, -1, -1, -1],
             [5, 3, 4, 3, 5, 4, 9, -1, 0, -1, -1],
             [5, 3, 4, 3, 5, 4, 9, -1, -1, -1, -1],
             [5, 3, 4, 4, 3, 5, 9, -1, -1, 0, -1],
             [5, 3, 4, 4, 3, 5, 9, 0, -1, -1, -1],
             [5, 3, 4, 4, 3, 5, 9, -1, 0, -1, -1],
             [5, 3, 4, 4, 3, 5, 9, -1, -1, -1, -1],
             [5, 3, 4, 4, 5, 3, 9, -1, -1, 0, -1],
             [5, 3, 4, 4, 5, 3, 9, 0, -1, -1, -1],
             [5, 3, 4, 4, 5, 3, 9, -1, 0, -1, -1],
             [5, 3, 4, 4, 5, 3, 9, -1, -1, -1, -1],
             [5, 3, 4, 5, 3, 4, 9, -1, -1, 0, -1],
             [5, 3, 4, 5, 3, 4, 9, 0, -1, -1, -1],
             [5, 3, 4, 5, 3, 4, 9, -1, 0, -1, -1],
             [5, 3, 4, 5, 3, 4, 9, -1, -1, -1, -1],
             [5, 3, 4, 5, 4, 3, 9, -1, -1, 0, -1],
             [5, 3, 4, 5, 4, 3, 9, 0, -1, -1, -1],
             [5, 3, 4, 5, 4, 3, 9, -1, 0, -1, -1],
             [5, 3, 4, 5, 4, 3, 9, -1, -1, -1, -1],
             [5, 4, 3, 3, 4, 5, 9, -1, -1, 0, -1],
             [5, 4, 3, 3, 4, 5, 9, 0, -1, -1, -1],
             [5, 4, 3, 3, 4, 5, 9, -1, 0, -1, -1],
             [5, 4, 3, 3, 4, 5, 9, -1, -1, -1, -1],
             [5, 4, 3, 3, 5, 4, 9, -1, -1, 0, -1],
             [5, 4, 3, 3, 5, 4, 9, 0, -1, -1, -1],
             [5, 4, 3, 3, 5, 4, 9, -1, 0, -1, -1],
             [5, 4, 3, 3, 5, 4, 9, -1, -1, -1, -1],
             [5, 4, 3, 4, 3, 5, 9, -1, -1, 0, -1],
             [5, 4, 3, 4, 3, 5, 9, 0, -1, -1, -1],
             [5, 4, 3, 4, 3, 5, 9, -1, 0, -1, -1],
             [5, 4, 3, 4, 3, 5, 9, -1, -1, -1, -1],
             [5, 4, 3, 4, 5, 3, 9, -1, -1, 0, -1],
             [5, 4, 3, 4, 5, 3, 9, 0, -1, -1, -1],
             [5, 4, 3, 4, 5, 3, 9, -1, 0, -1, -1],
             [5, 4, 3, 4, 5, 3, 9, -1, -1, -1, -1],
             [5, 4, 3, 5, 3, 4, 9, -1, -1, 0, -1],
             [5, 4, 3, 5, 3, 4, 9, 0, -1, -1, -1],
             [5, 4, 3, 5, 3, 4, 9, -1, 0, -1, -1],
             [5, 4, 3, 5, 3, 4, 9, -1, -1, -1, -1],
             [5, 4, 3, 5, 4, 3, 9, -1, -1, 0, -1],
             [5, 4, 3, 5, 4, 3, 9, 0, -1, -1, -1],
             [5, 4, 3, 5, 4, 3, 9, -1, 0, -1, -1],
             [5, 4, 3, 5, 4, 3, 9, -1, -1, -1, -1],
             [4, 5, 6, 4, 5, 6, 11, -1, -1, 0, -1],
             [4, 5, 6, 4, 5, 6, 11, 0, -1, -1, -1],
             [4, 5, 6, 4, 5, 6, 11, -1, 0, -1, -1],
             [4, 5, 6, 4, 5, 6, 11, -1, -1, -1, -1],
             [4, 5, 6, 4, 6, 5, 11, -1, -1, 0, -1],
             [4, 5, 6, 4, 6, 5, 11, 0, -1, -1, -1],
             [4, 5, 6, 4, 6, 5, 11, -1, 0, -1, -1],
             [4, 5, 6, 4, 6, 5, 11, -1, -1, -1, -1],
             [4, 5, 6, 5, 4, 6, 11, -1, -1, 0, -1],
             [4, 5, 6, 5, 4, 6, 11, 0, -1, -1, -1],
             [4, 5, 6, 5, 4, 6, 11, -1, 0, -1, -1],
             [4, 5, 6, 5, 4, 6, 11, -1, -1, -1, -1],
             [4, 5, 6, 5, 6, 4, 11, -1, -1, 0, -1],
             [4, 5, 6, 5, 6, 4, 11, 0, -1, -1, -1],
             [4, 5, 6, 5, 6, 4, 11, -1, 0, -1, -1],
             [4, 5, 6, 5, 6, 4, 11, -1, -1, -1, -1],
             [4, 5, 6, 6, 4, 5, 11, -1, -1, 0, -1],
             [4, 5, 6, 6, 4, 5, 11, 0, -1, -1, -1],
             [4, 5, 6, 6, 4, 5, 11, -1, 0, -1, -1],
             [4, 5, 6, 6, 4, 5, 11, -1, -1, -1, -1],
             [4, 5, 6, 6, 5, 4, 11, -1, -1, 0, -1],
             [4, 5, 6, 6, 5, 4, 11, 0, -1, -1, -1],
             [4, 5, 6, 6, 5, 4, 11, -1, 0, -1, -1],
             [4, 5, 6, 6, 5, 4, 11, -1, -1, -1, -1],
             [4, 6, 5, 4, 5, 6, 11, -1, -1, 0, -1],
             [4, 6, 5, 4, 5, 6, 11, 0, -1, -1, -1],
             [4, 6, 5, 4, 5, 6, 11, -1, 0, -1, -1],
             [4, 6, 5, 4, 5, 6, 11, -1, -1, -1, -1],
             [4, 6, 5, 4, 6, 5, 11, -1, -1, 0, -1],
             [4, 6, 5, 4, 6, 5, 11, 0, -1, -1, -1],
             [4, 6, 5, 4, 6, 5, 11, -1, 0, -1, -1],
             [4, 6, 5, 4, 6, 5, 11, -1, -1, -1, -1],
             [4, 6, 5, 5, 4, 6, 11, -1, -1, 0, -1],
             [4, 6, 5, 5, 4, 6, 11, 0, -1, -1, -1],
             [4, 6, 5, 5, 4, 6, 11, -1, 0, -1, -1],
             [4, 6, 5, 5, 4, 6, 11, -1, -1, -1, -1],
             [4, 6, 5, 5, 6, 4, 11, -1, -1, 0, -1],
             [4, 6, 5, 5, 6, 4, 11, 0, -1, -1, -1],
             [4, 6, 5, 5, 6, 4, 11, -1, 0, -1, -1],
             [4, 6, 5, 5, 6, 4, 11, -1, -1, -1, -1],
             [4, 6, 5, 6, 4, 5, 11, -1, -1, 0, -1],
             [4, 6, 5, 6, 4, 5, 11, 0, -1, -1, -1],
             [4, 6, 5, 6, 4, 5, 11, -1, 0, -1, -1],
             [4, 6, 5, 6, 4, 5, 11, -1, -1, -1, -1],
             [4, 6, 5, 6, 5, 4, 11, -1, -1, 0, -1],
             [4, 6, 5, 6, 5, 4, 11, 0, -1, -1, -1],
             [4, 6, 5, 6, 5, 4, 11, -1, 0, -1, -1],
             [4, 6, 5, 6, 5, 4, 11, -1, -1, -1, -1],
             [5, 4, 6, 4, 5, 6, 11, -1, -1, 0, -1],
             [5, 4, 6, 4, 5, 6, 11, 0, -1, -1, -1],
             [5, 4, 6, 4, 5, 6 ,11, -1, 0, -1, -1],
             [5, 4, 6, 4, 5, 6, 11, -1, -1, -1, -1],
             [5, 4, 6, 4, 6, 5, 11, -1, -1, 0, -1],
             [5, 4, 6, 4, 6, 5, 11, 0, -1, -1, -1],
             [5, 4, 6, 4, 6, 5, 11, -1, 0, -1, -1],
             [5, 4, 6, 4, 6, 5, 11, -1, -1, -1, -1],
             [5, 4, 6, 5, 4, 6, 11, -1, -1, 0, -1],
             [5, 4, 6, 5, 4, 6, 11, 0, -1, -1, -1],
             [5, 4, 6, 5, 4, 6, 11, -1, 0, -1, -1],
             [5, 4, 6, 5, 4, 6, 11, -1, -1, -1, -1],
             [5, 4, 6, 5, 6, 4, 11, -1, -1, 0, -1],
             [5, 4, 6, 5, 6, 4, 11, 0, -1, -1, -1],
             [5, 4, 6, 5, 6, 4, 11, -1, 0, -1, -1],
             [5, 4, 6, 5, 6, 4, 11, -1, -1, -1, -1],
             [5, 4, 6, 6, 4, 5, 11, -1, -1, 0, -1],
             [5, 4, 6, 6, 4, 5, 11, 0, -1, -1, -1],
             [5, 4, 6, 6, 4, 5, 11, -1, 0, -1, -1],
             [5, 4, 6, 6, 4, 5, 11, -1, -1, -1, -1],
             [5, 4, 6, 6, 5, 4, 11, -1, -1, 0, -1],
             [5, 4, 6, 6, 5, 4, 11, 0, -1, -1, -1],
             [5, 4, 6, 6, 5, 4, 11, -1, 0, -1, -1],
             [5, 4, 6, 6, 5, 4, 11, -1, -1, -1, -1],
             [5, 6, 4, 4, 5, 6, 11, -1, -1, 0, -1],
             [5, 6, 4, 4, 5, 6, 11, 0, -1, -1, -1],
             [5, 6, 4, 4, 5, 6, 11, -1, 0, -1, -1],
             [5, 6, 4, 4, 5, 6, 11, -1, -1, -1, -1],
             [5, 6, 4, 4, 6, 5, 11, -1, -1, 0, -1],
             [5, 6, 4, 4, 6, 5, 11, 0, -1, -1, -1],
             [5, 6, 4, 4, 6, 5, 11, -1, 0, -1, -1],
             [5, 6, 4, 4, 6, 5, 11, -1, -1, -1, -1],
             [5, 6, 4, 5, 4, 6, 11, -1, -1, 0, -1],
             [5, 6, 4, 5, 4, 6, 11, 0, -1, -1, -1],
             [5, 6, 4, 5, 4, 6, 11, -1, 0, -1, -1],
             [5, 6, 4, 5, 4, 6, 11, -1, -1, -1, -1],
             [5, 6, 4, 5, 6, 4, 11, -1, -1, 0, -1],
             [5, 6, 4, 5, 6, 4, 11, 0, -1, -1, -1],
             [5, 6, 4, 5, 6, 4, 11, -1, 0, -1, -1],
             [5, 6, 4, 5, 6, 4, 11, -1, -1, -1, -1],
             [5, 6, 4, 6, 4, 5, 11, -1, -1, 0, -1],
             [5, 6, 4, 6, 4, 5, 11, 0, -1, -1, -1],
             [5, 6, 4, 6, 4, 5, 11, -1, 0, -1, -1],
             [5, 6, 4, 6, 4, 5, 11, -1, -1, -1, -1],
             [5, 6, 4, 6, 5, 4, 11, -1, -1, 0, -1],
             [5, 6, 4, 6, 5, 4, 11, 0, -1, -1, -1],
             [5, 6, 4, 6, 5, 4, 11, -1, 0, -1, -1],
             [5, 6, 4, 6, 5, 4, 11, -1, -1, -1, -1],
             [6, 4, 5, 4, 5, 6, 11, -1, -1, 0, -1],
             [6, 4, 5, 4, 5, 6, 11, 0, -1, -1, -1],
             [6, 4, 5, 4, 5, 6, 11, -1, 0, -1, -1],
             [6, 4, 5, 4, 5, 6, 11, -1, -1, -1, -1],
             [6, 4, 5, 4, 6, 5, 11, -1, -1, 0, -1],
             [6, 4, 5, 4, 6, 5, 11, 0, -1, -1, -1],
             [6, 4, 5, 4, 6, 5, 11, -1, 0, -1, -1],
             [6, 4, 5, 4, 6, 5, 11, -1, -1, -1, -1],
             [6, 4, 5, 5, 4, 6, 11, -1, -1, 0, -1],
             [6, 4, 5, 5, 4, 6, 11, 0, -1, -1, -1],
             [6, 4, 5, 5, 4, 6, 11, -1, 0, -1, -1],
             [6, 4, 5, 5, 4, 6, 11, -1, -1, -1, -1],
             [6, 4, 5, 5, 6, 4, 11, -1, -1, 0, -1],
             [6, 4, 5, 5, 6, 4, 11, 0, -1, -1, -1],
             [6, 4, 5, 5, 6, 4, 11, -1, 0, -1, -1],
             [6, 4, 5, 5, 6, 4, 11, -1, -1, -1, -1],
             [6, 4, 5, 6, 4, 5, 11, -1, -1, 0, -1],
             [6, 4, 5, 6, 4, 5, 11, 0, -1, -1, -1],
             [6, 4, 5, 6, 4, 5, 11, -1, 0, -1, -1],
             [6, 4, 5, 6, 4, 5, 11, -1, -1, -1, -1],
             [6, 4, 5, 6, 5, 4, 11, -1, -1, 0, -1],
             [6, 4, 5, 6, 5, 4, 11, 0, -1, -1, -1],
             [6, 4, 5, 6, 5, 4, 11, -1, 0, -1, -1],
             [6, 4, 5, 6, 5, 4, 11, -1, -1, -1, -1],
             [6, 5, 4, 4, 5, 6, 11, -1, -1, 0, -1],
             [6, 5, 4, 4, 5, 6, 11, 0, -1, -1, -1],
             [6, 5, 4, 4, 5, 6, 11, -1, 0, -1, -1],
             [6, 5, 4, 4, 5, 6, 11, -1, -1, -1, -1],
             [6, 5, 4, 4, 6, 5, 11, -1, -1, 0, -1],
             [6, 5, 4, 4, 6, 5, 11, 0, -1, -1, -1],
             [6, 5, 4, 4, 6, 5, 11, -1, 0, -1, -1],
             [6, 5, 4, 4, 6, 5, 11, -1, -1, -1, -1],
             [6, 5, 4, 5, 4, 6, 11, -1, -1, 0, -1],
             [6, 5, 4, 5, 4, 6, 11, 0, -1, -1, -1],
             [6, 5, 4, 5, 4, 6, 11, -1, 0, -1, -1],
             [6, 5, 4, 5, 4, 6, 11, -1, -1, -1, -1],
             [6, 5, 4, 5, 6, 4, 11, -1, -1, 0, -1],
             [6, 5, 4, 5, 6, 4, 11, 0, -1, -1, -1],
             [6, 5, 4, 5, 6, 4, 11, -1, 0, -1, -1],
             [6, 5, 4, 5, 6, 4, 11, -1, -1, -1, -1],
             [6, 5, 4, 6, 4, 5, 11, -1, -1, 0, -1],
             [6, 5, 4, 6, 4, 5, 11, 0, -1, -1, -1],
             [6, 5, 4, 6, 4, 5, 11, -1, 0, -1, -1],
             [6, 5, 4, 6, 4, 5, 11, -1, -1, -1, -1],
             [6, 5, 4, 6, 5, 4, 11, -1, -1, 0, -1],
             [6, 5, 4, 6, 5, 4, 11, 0, -1, -1, -1],
             [6, 5, 4, 6, 5, 4, 11, -1, 0, -1, -1],
             [6, 5, 4, 6, 5, 4, 11, -1, -1, -1, -1],
             [4, 5, 6, 6, 4, 5, 15, -1, -1, -1, -1],
             [4, 5, 6, 4, 6, 5, 6, -1, 0, -1, -1],
             [4, 5, 6, 5, 4, 6, 6, 0, -1, -1 ,-1],
             [4, 5, 6, 6, 4, 5, 15, 0, -1, -1, -1],
             [4, 6, 5, 6, 5, 4, 4, 0, -1, -1, -1],
             [4, 6, 5, 5, 4, 6, 5, -1, 0, -1, -1],
             [4, 6, 5, 6, 5, 4, 15, -1, -1, 0, -1],
             [4, 6, 5, 6, 4, 5, 6, -1, -1, -1, -1],
             [5, 4, 6, 4, 5, 6, 15, 0, -1, -1, -1],
             [5, 4, 6, 4, 6, 5, 15, -1, -1, 0, -1],
             [5, 4, 6, 6, 4, 5, 4, -1, -1, -1, -1],
             [5, 4, 6, 5, 6, 4, 4, -1, 0, -1, -1],
             [6, 5, 4, 5, 6, 4, 6, -1, -1, 0, -1],
             [6, 5, 4, 5, 4, 6, 4, -1, -1, -1, -1],
             [6, 5, 4, 6, 4, 5, 5, -1, -1, -1, -1],
             [6, 5, 4, 4, 5, 6, 15, -1, -1, 0, -1],
             [5, 6, 4, 5, 6, 4, 15, 0, -1, -1, -1],
             [5, 6, 4, 5, 4, 6, 15, -1, -1, -1, -1],
             [5, 6, 4, 6, 4, 5, 15, -1, 0, -1, -1],
             [5, 6, 4, 6, 5, 4, 15, -1, -1, 0, -1],
             [6, 4, 5, 4, 6, 5, 6, -1, -1, -1, -1],
             [6, 4, 5, 5, 4, 6, 6, 0, -1, -1, -1],
             [6, 4, 5, 6, 5, 4, 4, -1, -1, -1, -1],
             [6, 4, 5, 6, 4, 5, 6, -1, 0, -1, -1],
             [5, 6, 7, 5, 6, 7, 13, -1, -1, 0, -1],
             [5, 6, 7, 5, 6, 7, 13, 0, -1, -1, -1],
             [5, 6, 7, 5, 6, 7, 13, -1, 0, -1, -1],
             [5, 6, 7, 5, 6, 7, 13, -1, -1, -1, -1],
             [5, 6, 7, 5, 7, 6, 13, -1, -1, 0, -1],
             [5, 6, 7, 5, 7, 6, 13, 0, -1, -1, -1],
             [5, 6, 7, 5, 7, 6, 13, -1, 0, -1, -1],
             [5, 6, 7, 5, 7, 6, 13, -1, -1, -1, -1],
             [5, 6, 7, 6, 5, 7, 13, -1, -1, 0, -1],
             [5, 6, 7, 6, 5, 7, 13, 0, -1, -1, -1],
             [5, 6, 7, 6, 5, 7, 13, -1, 0, -1, -1],
             [5, 6, 7, 6, 5, 7, 13, -1, -1, -1, -1],
             [5, 6, 7, 6, 7, 5, 13, -1, -1, 0, -1],
             [5, 6, 7, 6, 7, 5, 13, 0, -1, -1, -1],
             [5, 6, 7, 6, 7, 5, 13, -1, 0, -1, -1],
             [5, 6, 7, 6, 7, 5, 13, -1, -1, -1, -1],
             [5, 6, 7, 7, 5, 6, 13, -1, -1, 0, -1],
             [5, 6, 7, 7, 5, 6, 13, 0, -1, -1, -1],
             [5, 6, 7, 7, 5, 6, 13, -1, 0, -1, -1],
             [5, 6, 7, 7, 5, 6, 13, -1, -1, -1, -1],
             [5, 6, 7, 7, 6, 5, 13, -1, -1, 0, -1],
             [5, 6, 7, 7, 6, 5, 13, 0, -1, -1, -1],
             [5, 6, 7, 7, 6, 5, 13, -1, 0, -1, -1],
             [5, 6, 7, 7, 6, 5, 13, -1, -1, -1, -1],
             [5, 7, 6, 5, 6, 7, 13, -1, -1, 0, -1],
             [5, 7, 6, 5, 6, 7, 13, 0, -1, -1, -1],
             [5, 7, 6, 5, 6, 7, 13, -1, 0, -1, -1],
             [5, 7, 6, 5, 6, 7, 13, -1, -1, -1, -1],
             [5, 7, 6, 5, 7, 6, 13, -1, -1, 0, -1],
             [5, 7, 6, 5, 7, 6, 13, 0, -1, -1, -1],
             [5, 7, 6, 5, 7, 6, 13, -1, 0, -1, -1],
             [5, 7, 6, 5, 7, 6, 13, -1, -1, -1, -1],
             [5, 7, 6, 6, 5, 7, 13, -1, -1, 0, -1],
             [5, 7, 6, 6, 5, 7, 13, 0, -1, -1, -1],
             [5, 7, 6, 6, 5 ,7, 13, -1, 0, -1, -1],
             [5, 7, 6, 6, 5, 7, 13, -1, -1, -1, -1],
             [5, 7, 6, 6, 7, 5, 13, -1, -1, 0, -1],
             [5, 7, 6, 6, 7, 5, 13, 0, -1, -1, -1],
             [5, 7, 6, 6, 7, 5, 13, -1, 0, -1, -1],
             [5, 7, 6, 6, 7, 5, 13, -1, -1, -1, -1],
             [5, 7, 6, 7, 5, 6, 13, -1, -1, 0, -1],
             [5, 7, 6, 7, 5, 6, 13, 0, -1, -1, -1],
             [5, 7, 6, 7, 5, 6, 13, -1, 0, -1, -1],
             [5, 7, 6, 7, 5, 6, 13, -1, -1, -1, -1],
             [5, 7, 6, 7, 6, 5, 13, -1, -1, 0, -1],
             [5, 7, 6, 7, 6, 5 ,13, 0, -1, -1, -1],
             [5, 7, 6, 7, 6, 5, 13, -1, 0, -1, -1],
             [5, 7, 6, 7, 6, 5, 13, -1, -1, -1, -1],
             [6, 5, 7, 5, 6, 7, 13, -1, -1, 0, -1],
             [6, 5, 7, 5, 6, 7, 13, 0, -1, -1, -1],
             [6, 5, 7, 5, 6, 7, 13, -1, 0, -1, -1],
             [6, 5, 7, 5, 6, 7, 13, -1, -1, -1, -1],
             [6, 5, 7, 5, 7, 6, 13, -1, -1, 0, -1],
             [6, 5, 7, 5, 7, 6, 13, 0, -1, -1, -1],
             [6, 5, 7, 5, 7, 6, 13, -1, 0, -1, -1],
             [6, 5, 7, 5, 7, 6, 13, -1, -1, -1, -1],
             [6, 5, 7, 6, 5, 7, 13, -1, -1, 0, -1],
             [6, 5, 7, 6, 5, 7, 13, 0, -1, -1, -1],
             [6, 5, 7, 6, 5, 7, 13, -1, 0, -1, -1],
             [6, 5, 7, 6, 5, 7, 13, -1, -1, -1, -1],
             [6, 5, 7, 6, 7, 5, 13, -1, -1, 0, -1],
             [6, 5, 7, 6, 7, 5, 13, 0, -1, -1, -1],
             [6, 5, 7, 6, 7, 5, 13, -1, 0, -1, -1],
             [6, 5, 7, 6, 7, 5, 13, -1, -1, -1, -1],
             [6, 5, 7, 7, 5, 6, 13, -1, -1, 0, -1],
             [6, 5, 7, 7, 5, 6, 13, 0, -1, -1, -1],
             [6, 5, 7, 7, 5, 6, 13, -1, 0, -1, -1],
             [6, 5, 7, 7, 5, 6, 13, -1, -1, -1, -1],
             [6, 5, 7, 7, 6, 5, 13, -1, -1, 0, -1],
             [6, 5, 7, 7, 6, 5, 13, 0, -1, -1, -1],
             [6, 5, 7, 7, 6, 5, 13, -1, 0, -1, -1],
             [6, 5, 7, 7, 6, 5, 13, -1, -1, -1, -1],
             [6, 7, 5, 5, 6, 7, 13, -1, -1 ,0, -1],
             [6, 7, 5, 5, 6, 7, 13, 0, -1, -1, -1],
             [6, 7, 5, 5, 6, 7, 13, -1, 0, -1, -1],
             [6, 7, 5, 5, 6, 7, 13, -1, -1, -1, -1],
             [6, 7, 5, 5, 7, 6, 13, -1, -1, 0, -1],
             [6, 7, 5, 5, 7, 6, 13, 0, -1, -1, -1],
             [6, 7, 5, 5, 7, 6, 13, -1, 0, -1, -1],
             [6, 7, 5, 5, 7, 6, 13, -1, -1, -1, -1],
             [6, 7, 5, 6, 5, 7, 13, -1, -1, 0, -1],
             [6, 7, 5, 6, 5, 7, 13, 0, -1, -1, -1],
             [6, 7, 5, 6, 5, 7, 13, -1, 0, -1, -1],
             [6, 7, 5, 6, 5, 7, 13, -1, -1, -1, -1],
             [6, 7, 5, 6, 7, 5, 13, -1, -1, 0, -1],
             [6, 7, 5, 6, 7, 5, 13, 0, -1, -1, -1],
             [6, 7, 5, 6, 7, 5, 13, -1, 0, -1, -1],
             [6, 7, 5, 6, 7, 5, 13, -1, -1, -1, -1],
             [6, 7, 5, 7, 5, 6, 13, -1, -1, 0, -1],
             [6, 7, 5, 7, 5, 6, 13, 0, -1, -1, -1],
             [6, 7, 5, 7, 5, 6, 13, -1, 0, -1, -1],
             [6, 7, 5, 7, 5, 6, 13, -1, -1, -1, -1],
             [6, 7, 5, 7, 6, 5, 13, -1, -1, 0, -1],
             [6, 7, 5, 7, 6, 5, 13, 0, -1, -1, -1],
             [6, 7, 5, 7, 6, 5, 13, -1, 0, -1, -1],
             [6, 7, 5, 7, 6, 5, 13, -1, -1, -1, -1],
             [7, 5, 6, 5, 6, 7, 13, -1, -1, 0, -1],
             [7, 5, 6, 5, 6, 7, 13, 0, -1, -1, -1],
             [7, 5, 6, 5, 6, 7, 13, -1, 0, -1, -1],
             [7, 5, 6, 5, 6, 7, 13, -1, -1, -1, -1],
             [7, 5, 6, 5, 7, 6, 13, -1, -1, 0, -1],
             [7, 5, 6, 5, 7, 6, 13, 0, -1, -1, -1],
             [7, 5, 6, 5, 7, 6, 13, -1, 0, -1, -1],
             [7, 5, 6, 5, 7, 6, 13, -1, -1, -1, -1],
             [7, 5, 6, 6, 5, 7, 13, -1, -1, 0, -1],
             [7, 5, 6, 6, 5, 7, 13, 0, -1, -1, -1],
             [7, 5, 6, 6, 5, 7, 13, -1, 0, -1, -1],
             [7, 5, 6, 6, 5, 7, 13, -1, -1, -1, -1],
             [7, 5, 6, 6, 7, 5, 13, -1, -1, 0, -1],
             [7, 5, 6, 6, 7, 5, 13, 0, -1, -1, -1],
             [7, 5, 6, 6, 7, 5, 13, -1, 0, -1, -1],
             [7, 5, 6, 6, 7, 5, 13, -1, -1, -1, -1],
             [7, 5, 6, 7, 5, 6, 13, -1, -1, 0, -1],
             [7, 5, 6, 7, 5, 6, 13, 0, -1, -1, -1],
             [7, 5, 6, 7, 5, 6, 13, -1, 0, -1, -1],
             [7, 5, 6, 7, 5, 6, 13, -1, -1, -1, -1],
             [7, 5, 6, 7, 6, 5, 13, -1, -1, 0, -1],
             [7, 5, 6, 7, 6, 5, 13, 0, -1, -1, -1],
             [7, 5, 6, 7, 6, 5, 13, -1, 0, -1, -1],
             [7, 5, 6, 7, 6, 5, 13, -1, -1, -1, -1],
             [7, 6, 5, 5, 6, 7, 13, -1, -1 ,0, -1],
             [7, 6, 5, 5, 6, 7, 13, 0, -1, -1, -1],
             [7, 6, 5, 5, 6, 7, 13, -1, 0, -1, -1],
             [7, 6, 5, 5, 6, 7, 13, -1, -1, -1, -1],
             [7, 6, 5, 5, 7, 6, 13, -1, -1 ,0, -1],
             [7, 6, 5, 5, 7, 6, 13, 0, -1, -1, -1],
             [7, 6, 5, 5, 7, 6, 13, -1, 0, -1, -1],
             [7, 6, 5, 5, 7, 6, 13, -1, -1, -1, -1],
             [7, 6, 5, 6, 5, 7, 13, -1, -1, 0, -1],
             [7, 6, 5, 6, 5, 7, 13, 0, -1, -1, -1],
             [7, 6, 5, 6, 5, 7, 13, -1, 0, -1, -1],
             [7, 6, 5, 6, 5, 7, 13, -1, -1, -1, -1],
             [7, 6, 5, 6, 7, 5, 13, -1, -1, 0, -1],
             [7, 6, 5, 6, 7, 5, 13, 0, -1, -1, -1],
             [7, 6, 5, 6, 7, 5, 13, -1, 0, -1, -1],
             [7, 6, 5, 6, 7, 5, 13, -1, -1, -1, -1],
             [7, 6, 5, 7, 5, 6, 13, -1, -1, 0, -1],
             [7, 6, 5, 7, 5, 6, 13, 0, -1 ,-1, -1],
             [7, 6, 5, 7, 5, 6, 13, -1, 0, -1, -1],
             [7, 6, 5, 7, 5, 6, 13, -1, -1, -1, -1],
             [7, 6, 5, 7, 6, 5, 13, -1, -1, 0, -1],
             [7, 6, 5, 7, 6, 5, 13, 0, -1, -1, -1],
             [7, 6, 5, 7, 6, 5, 13, -1, 0, -1, -1],
             [7, 6, 5, 7, 6, 5, 13, -1, -1, -1, -1],
             [5, 6, 7, 7, 6, 5, 18, -1, 0, -1, -1],
             [5, 6, 7, 6, 5, 7, 7, 0, -1, -1, -1],
             [5, 6, 7, 5, 7, 6, 6, -1, -1, -1, -1],
             [5, 6, 7, 5, 7, 6, 18, -1, -1, 0, -1],
             [7, 5, 6, 6, 5, 7, 6, -1, -1, 0, -1],
             [7, 5, 6, 6, 7, 5, 7, 0, -1, -1, -1],
             [7, 5, 6, 5, 6, 7, 7, -1, -1, -1, -1],
             [7, 5, 6, 7, 5, 6, 5, -1, -1, 0, -1],
             [6, 5, 7, 6, 7, 5, 18, -1, -1, 0, -1],
             [6, 5, 7, 6, 5, 7, 18, 0, -1, -1, -1],
             [6, 5, 7, 7, 6, 5, 18, -1, -1, -1, -1],
             [6, 5, 7, 5, 7, 6, 18, -1, 0, -1, -1],
             [7, 6, 5, 7, 6, 5, 7, -1, -1, 0, -1],
             [7, 6, 5, 7, 5, 6, 7, -1, -1, -1, -1],
             [7, 6, 5, 6, 7, 5, 5, -1, 0, -1, -1],
             [7, 6, 5, 5, 6, 7, 6, -1, -1, 0, -1],
             [5, 7, 6, 5, 6, 7, 18, 0, -1, -1, -1],
             [5, 7, 6, 6, 7, 5, 18, 0, -1, -1, -1],
             [5, 7, 6, 6, 5, 7, 6, -1, -1, -1, -1],
             [5, 7, 6, 5, 7, 6, 7, 0, -1, -1, -1],
             [6, 7, 5, 5, 7, 6, 6, -1, 0, -1, -1],
             [6, 7, 5, 5, 6, 7, 5, -1, -1, -1, -1],
             [6, 7, 5, 7, 6, 5, 18, -1, -1, 0, -1],
             [6, 7, 5, 6, 7, 5, 6, -1, -1, -1, -1],
             [6, 7, 8, 6, 7, 8, 15, -1, -1, 0, -1],
             [6, 7, 8, 6, 7, 8, 15, 0, -1, -1, -1],
             [6, 7, 8, 6, 7, 8, 15, -1, 0, -1, -1],
             [6, 7, 8, 6, 7, 8, 15, -1, -1, -1, -1],
             [6, 7, 8, 6, 8, 7, 15, -1, -1, 0, -1],
             [6, 7, 8, 6, 8, 7, 15, 0, -1, -1, -1],
             [6, 7, 8, 6, 8, 7, 15, -1, 0, -1, -1],
             [6, 7, 8, 6, 8, 7, 15, -1, -1, -1, -1],
             [6, 7, 8, 7, 6, 8, 15, -1, -1, 0, -1],
             [6, 7, 8, 7, 6, 8, 15, 0, -1, -1, -1],
             [6, 7, 8, 7, 6, 8, 15, -1, 0, -1, -1],
             [6, 7, 8, 7, 6, 8, 15, -1, -1, -1, -1],
             [6, 7, 8, 7, 8, 6, 15, -1, -1, 0, -1],
             [6, 7, 8, 7, 8, 6, 15, 0, -1, -1, -1],
             [6, 7, 8, 7, 8, 6, 15, -1, 0, -1, -1],
             [6, 7, 8, 7, 8, 6, 15, -1, -1, -1, -1],
             [6, 7, 8, 8, 6, 7, 15, -1, -1, 0, -1],
             [6, 7, 8, 8, 6, 7, 15, 0, -1, -1, -1],
             [6, 7, 8, 8, 6, 7, 15, -1, 0, -1, -1],
             [6, 7, 8, 8, 6, 7, 15, -1, -1, -1, -1],
             [6, 7, 8, 8, 7, 6, 15, -1, -1, 0, -1],
             [6, 7, 8, 8, 7, 6, 15, 0, -1, -1, -1],
             [6, 7, 8, 8, 7, 6, 15, -1, 0, -1, -1],
             [6, 7, 8, 8, 7, 6, 15, -1, -1, -1, -1],
             [6, 8, 7, 6, 7, 8, 15, -1, -1, 0, -1],
             [6, 8, 7, 6, 7, 8, 15, 0, -1, -1, -1],
             [6, 8, 7, 6, 7, 8, 15, -1, 0, -1, -1],
             [6, 8, 7, 6, 7, 8, 15, -1, -1, -1, -1],
             [6, 8, 7, 6, 8, 7, 15, -1, -1, 0, -1],
             [6, 8, 7, 6, 8, 7, 15, 0, -1, -1, -1],
             [6, 8, 7, 6, 8, 7, 15, -1, 0, -1, -1],
             [6, 8, 7, 6, 8, 7, 15, -1, -1, -1, -1],
             [6, 8, 7, 7, 6, 8, 15, -1, -1, 0, -1],
             [6, 8, 7, 7, 6, 8, 15, 0, -1, -1, -1],
             [6, 8, 7, 7, 6, 8, 15, -1, 0, -1, -1],
             [6, 8, 7, 7, 6, 8, 15, -1, -1, -1, -1],
             [6, 8, 7, 7, 8, 6, 15, -1, -1, 0, -1],
             [6, 8, 7, 7, 8, 6, 15, 0, -1, -1, -1],
             [6, 8, 7, 7, 8, 6, 15, -1, 0, -1 ,-1],
             [6, 8, 7, 7, 8, 6, 15, -1, -1, -1, -1],
             [6, 8, 7, 8, 6, 7, 15, -1, -1, 0, -1],
             [6, 8, 7, 8, 6, 7, 15, 0, -1, -1, -1],
             [6, 8, 7, 8, 6, 7, 15, -1, 0, -1, -1],
             [6, 8, 7, 8, 6, 7, 15, -1, -1, -1, -1],
             [6, 8, 7, 8, 7, 6, 15, -1, -1, 0, -1],
             [6, 8, 7, 8, 7, 6, 15, 0, -1, -1, -1],
             [6, 8, 7, 8, 7, 6, 15, -1, 0, -1 ,-1],
             [6, 8, 7, 8, 7, 6, 15, -1, -1, -1, -1],
             [7, 6, 8, 6, 7, 8, 15, -1, -1, 0, -1],
             [7, 6, 8, 6, 7, 8, 15, 0, -1, -1, -1],
             [7, 6, 8, 6, 7, 8, 15, -1, 0, -1, -1],
             [7, 6, 8, 6, 7, 8, 15, -1, -1, -1, -1],
             [7, 6, 8, 6, 8, 7, 15, -1, -1, 0, -1],
             [7, 6, 8, 6, 8, 7, 15, 0, -1, -1, -1],
             [7, 6, 8, 6, 8, 7, 15, -1, 0, -1, -1],
             [7, 6, 8, 6, 8, 7, 15, -1, -1, -1, -1],
             [7, 6, 8, 7, 6, 8, 15, -1, -1, 0, -1],
             [7, 6, 8, 7, 6, 8, 15, 0, -1, -1, -1],
             [7, 6, 8, 7, 6, 8, 15, -1, 0, -1, -1],
             [7, 6, 8, 7, 6, 8, 15, -1, -1, -1, -1],
             [7, 6, 8, 7, 8, 6, 15, -1, -1, 0, -1],
             [7, 6, 8, 7, 8, 6, 15, 0, -1, -1, -1],
             [7, 6, 8, 7, 8, 6, 15, -1, 0, -1, -1],
             [7, 6, 8, 7, 8, 6, 15, -1, -1, -1, -1],
             [7, 6, 8, 8, 6, 7, 15, -1, -1, 0, -1],
             [7, 6, 8, 8, 6, 7, 15, 0, -1, -1, -1],
             [7, 6, 8, 8, 6, 7, 15, -1, 0, -1, -1],
             [7, 6, 8, 8, 6, 7, 15, -1, -1, -1, -1],
             [7, 6, 8, 8, 7, 6, 15, -1, -1, 0, -1],
             [7, 6, 8, 8, 7, 6, 15, 0, -1, -1, -1],
             [7, 6, 8, 8, 7, 6, 15, -1, 0, -1, -1],
             [7, 6, 8, 8, 7, 6, 15, -1, -1, -1, -1],
             [7, 8, 6, 6, 7, 8, 15, -1, -1, 0, -1],
             [7, 8, 6, 6, 7, 8, 15, 0, -1, -1, -1],
             [7, 8, 6, 6, 7, 8, 15, -1, 0, -1, -1],
             [7, 8, 6, 6, 7, 8, 15, -1, -1, -1, -1],
             [7, 8, 6, 6, 8, 7, 15, -1, -1, 0, -1],
             [7, 8, 6, 6, 8, 7, 15, 0, -1, -1, -1],
             [7, 8, 6, 6, 8, 7, 15, -1, 0, -1, -1],
             [7, 8, 6, 6, 8, 7, 15, -1, -1, -1, -1],
             [7, 8, 6, 7, 6, 8, 15, -1, -1, 0, -1],
             [7, 8, 6, 7, 6, 8, 15, 0, -1, -1, -1],
             [7, 8, 6, 7, 6, 8, 15, -1, 0, -1, -1],
             [7, 8, 6, 7, 6, 8, 15, -1, -1, -1, -1],
             [7, 8, 6, 7, 8, 6, 15, -1, -1, 0, -1],
             [7, 8, 6, 7, 8, 6, 15, 0, -1, -1, -1],
             [7, 8, 6, 7, 8, 6, 15, -1, 0, -1, -1],
             [7, 8, 6, 7, 8, 6, 15, -1, -1, -1, -1],
             [7, 8, 6, 8, 6, 7, 15, -1, -1, 0, -1],
             [7, 8, 6, 8, 6, 7, 15, 0, -1, -1, -1],
             [7, 8, 6, 8, 6, 7, 15, -1, 0, -1, -1],
             [7, 8, 6, 8, 6, 7, 15, -1, -1, -1, -1],
             [7, 8, 6, 8, 7, 6, 15, -1, -1, 0, -1],
             [7, 8, 6, 8, 7, 6, 15, 0, -1, -1, -1],
             [7, 8, 6, 8, 7, 6, 15, -1, 0, -1, -1],
             [7, 8, 6, 8, 7, 6, 15, -1, -1, -1, -1],
             [8, 6, 7, 6, 7, 8, 15, -1, -1, 0, -1],
             [8, 6, 7, 6, 7, 8, 15, 0, -1, -1, -1],
             [8, 6, 7, 6, 7, 8, 15, -1, 0, -1, -1],
             [8, 6, 7, 6, 7, 8, 15, -1, -1, -1, -1],
             [8, 6, 7, 6, 8, 7, 15, -1, -1, 0, -1],
             [8, 6, 7, 6, 8, 7, 15, 0, -1, -1, -1],
             [8, 6, 7, 6, 8, 7, 15, -1, 0, -1, -1],
             [8, 6, 7, 6, 8, 7, 15, -1, -1, -1, -1],
             [8, 6, 7, 7, 6, 8, 15, -1, -1, 0, -1],
             [8, 6, 7, 7, 6, 8, 15, 0, -1, -1, -1],
             [8, 6, 7, 7, 6, 8, 15, -1, 0, -1, -1],
             [8, 6, 7, 7, 6, 8, 15, -1, -1, -1, -1],
             [8, 6, 7, 7, 8, 6, 15, -1, -1, 0, -1],
             [8, 6, 7, 7, 8, 6, 15, 0, -1, -1, -1],
             [8, 6, 7, 7, 8, 6, 15, -1, 0, -1, -1],
             [8, 6, 7, 7, 8, 6, 15, -1, -1, -1, -1],
             [8, 6, 7, 8, 6, 7, 15, -1, -1, 0, -1],
             [8, 6, 7, 8, 6, 7, 15, 0, -1, -1, -1],
             [8, 6, 7, 8, 6, 7, 15, -1, 0, -1, -1],
             [8, 6, 7, 8, 6, 7, 15, -1, -1, -1, -1],
             [8, 6, 7, 8, 7, 6, 15, -1, -1, 0, -1],
             [8, 6, 7, 8, 7, 6, 15, 0, -1, -1, -1],
             [8, 6, 7, 8, 7, 6, 15, -1, 0, -1, -1],
             [8, 6, 7, 8, 7, 6, 15, -1, -1, -1, -1],
             [8, 7, 6, 6, 7, 8, 15, -1, -1, 0, -1],
             [8, 7, 6, 6, 7, 8, 15, 0, -1, -1, -1],
             [8, 7, 6, 6, 7, 8, 15, -1, 0, -1, -1],
             [8, 7, 6, 6, 7, 8, 15, -1, -1, -1, -1],
             [8, 7, 6, 6, 8, 7, 15, -1, -1, 0, -1],
             [8, 7, 6, 6, 8, 7, 15, 0, -1, -1, -1],
             [8, 7, 6, 6, 8, 7, 15, -1, 0, -1, -1],
             [8, 7, 6, 6, 8, 7, 15, -1, -1, -1, -1],
             [8, 7, 6, 7, 6, 8, 15, -1, -1, 0, -1],
             [8, 7, 6, 7, 6, 8, 15, 0, -1, -1, -1],
             [8, 7, 6, 7, 6, 8, 15, -1, 0, -1, -1],
             [8, 7, 6, 7, 6, 8, 15, -1, -1, -1, -1],
             [8, 7, 6, 7, 8, 6, 15, -1, -1, 0, -1],
             [8, 7, 6, 7, 8, 6, 15, 0, -1, -1, -1],
             [8, 7, 6, 7, 8, 6, 15, -1, 0, -1, -1],
             [8, 7, 6, 7, 8, 6, 15, -1, -1, -1, -1],
             [8, 7, 6, 8, 6, 7, 15, -1, -1, 0, -1],
             [8, 7, 6, 8, 6, 7, 15, 0, -1, -1, -1],
             [8, 7, 6, 8, 6, 7, 15, -1, 0, -1, -1],
             [8, 7, 6, 8, 6, 7, 15, -1, -1, -1, -1],
             [8, 7, 6, 8, 7, 6, 15, -1, -1, 0, -1],
             [8, 7, 6, 8, 7, 6, 15, 0, -1, -1, -1],
             [8, 7, 6, 8, 7, 6, 15, -1, 0, -1, -1],
             [8, 7, 6, 8, 7, 6, 15, -1, -1, -1, -1],
             [7, 6, 8, 6, 7, 8, 21, 0, -1, -1, -1],
             [7, 6, 8, 6, 8, 7, 21, -1, -1, 0, -1],
             [7, 6, 8, 8, 6, 7, 6, -1, -1, -1, -1],
             [7, 6, 8, 7, 8, 6, 7, -1, 0, -1, -1],
             [6, 7, 8, 8, 7, 6, 8, -1, -1, -1, -1],
             [6, 7, 8, 7, 6, 8, 8, -1, 0, -1, -1],
             [6, 7, 8, 6, 8, 7, 7, -1, -1, 0, -1],
             [6, 7, 8, 6, 7, 8, 7, 0, -1, -1, -1],
             [8, 7, 6, 8, 7, 6, 13, -1, 0, -1, -1],
             [8, 7, 6, 8, 7, 6, 8, 0, -1, -1, -1],
             [8, 7, 6, 8, 6, 7, 8, -1, -1, -1, -1],
             [8, 7, 6, 8, 6, 7, 7, -1, -1, 0, -1],
             [8, 6, 7, 6, 8, 7, 21, 0, -1, -1, -1],
             [8, 6, 7, 8, 7, 6, 21, -1, 0, -1, -1],
             [8, 6, 7, 8, 7, 6, 25, -1, -1, -1, -1],
             [8, 6, 7, 7, 8, 6, 28, -1, -1, -1, -1],
             [7, 8, 6, 8, 7, 6, 21, -1, -1, 0, -1],
             [7, 8, 6, 7, 6, 8, 21, -1, -1, -1, -1],
             [7, 8, 6, 7, 8, 6, 21, -1, 0, -1, -1],
             [7, 8, 6, 7, 8, 6, 21, 0, -1, -1, -1],
             [6, 8, 7, 8, 6, 7, 7, 0, -1, -1, -1],
             [6, 8, 7, 8, 7, 6, 8, -1, 0, -1, -1],
             [6, 8, 7, 8, 7, 6, 6, -1, -1, -1, -1],
             [6, 8, 7, 7, 8, 6, 8, -1, 0, -1, -1],
             [2, 8, 9, 8, 2, 9, 19, -1, -1, 0, -1],
             [2, 8, 9, 8, 2, 9, 19, 0, -1, -1, -1],
             [2, 8, 9, 8, 2, 9, 19, -1, 0, -1, -1],
             [2, 8, 9, 8, 2, 9, 19, -1, -1, -1, -1],
             [2, 8, 9, 2, 8, 9, 19, -1, -1, 0, -1],
             [2, 8, 9, 2, 8, 9, 19, 0, -1, -1, -1],
             [2, 8, 9, 2, 8, 9, 19, -1, 0, -1, -1],
             [2, 8, 9, 2, 8, 9, 19, -1, -1, -1, -1],
             [2, 8, 9, 9, 2, 8, 19, -1, -1, 0, -1],
             [2, 8, 9, 9, 2, 8, 19, 0, -1, -1, -1],
             [2, 8, 9, 9, 2, 8, 19, -1, 0, -1, -1],
             [2, 8, 9, 9, 2, 8, 19, -1, -1, -1, -1],
             [2, 8, 9, 2, 9, 8, 19, -1, -1, 0, -1],
             [2, 8, 9, 2, 9, 8, 19, 0, -1, -1, -1],
             [2, 8, 9, 2, 9, 8, 19, -1, 0, -1, -1],
             [2, 8, 9, 2, 9, 8, 19, -1, -1, -1, -1],
             [2, 8, 9, 8, 9, 2, 19, -1, -1, 0, -1],
             [2, 8, 9, 8, 9, 2, 19, 0, -1, -1, -1],
             [2, 8, 9, 8, 9, 2, 19, -1, 0, -1, -1],
             [2, 8, 9, 8, 9, 2, 19, -1, -1, -1, -1],
             [2, 8, 9, 9, 8, 2, 19, -1, -1, 0, -1],
             [2, 8, 9, 9, 8, 2, 19, 0, -1, -1, -1],
             [2, 8, 9, 9, 8, 2, 19, -1, 0, -1, -1],
             [2, 8, 9, 9, 8, 2, 19, -1, -1, -1, -1],
             [9, 8, 3, 3, 8, 9, 8, -1, -1, 0, -1],
             [9, 8, 3, 3, 8, 9, 8, 0, -1, -1, -1],
             [9, 8, 3, 3, 8, 9, 9, -1, -1, -1, -1],
             [9, 8, 3, 3, 8, 9, 9, -1, 0, -1, -1],
             [9, 8, 3, 3, 9, 8, 8, -1, 0, -1, -1],
             [9, 8, 3, 3, 9, 8, 8, 0, -1, -1, -1],
             [9, 8, 3, 3, 9, 8, 8, -1, -1, -1, -1],
             [9, 8, 3, 3, 9, 8, 8, -1, -1, 0, -1],
             [9, 8, 3, 8, 3, 9, 9, -1, -1, 0, -1],
             [9, 8, 3, 8, 3, 9, 9, -1, -1, -1, -1],
             [9, 8, 3, 8, 3, 9, 8, -1, -1, 0, -1],
             [9, 8, 3, 8, 3, 9, 8, 0, -1, -1, -1],
             [9, 8, 3, 8, 9, 3, 8, -1, 0, -1, -1],
             [9, 8, 3, 8, 9, 3, 9, -1, 0, -1, -1],
             [9, 8, 3, 8, 9, 3, 3, 0, -1, -1, -1],
             [9, 8, 3, 8, 9, 3, 8, 0, -1, -1, -1],
             [9, 8, 3, 9, 3, 8, 9, -1, -1, -1, -1],
             [9, 8, 3, 9, 3, 8, 3, -1, 0, -1, -1],
             [9, 8, 3, 9, 3, 8, 8, -1, -1, -1, -1],
             [9, 8, 3, 9, 3, 8, 8, 0, -1, -1, -1],
             [9, 8, 3, 9, 8, 3, 3, -1, 0, -1, -1],
             [9, 8, 3, 9, 8, 3, 3, -1, -1, -1, -1],
             [9, 8, 3, 9, 8, 3, 3, 0, -1, -1, -1],
             [9, 8, 3, 9, 8, 3, 9, -1, -1, 0, -1],
             [9, 5, 7, 5, 7, 9, 7, -1, -1, 0, -1],
             [9, 5, 7, 5, 7, 9, 7, 0, -1, -1, -1],
             [9, 5, 7, 5, 7, 9, 7, -1, 0, -1, -1],
             [9, 5, 7, 5, 7, 9, 9, -1, 0, -1, -1],
             [9, 5, 7, 5, 9, 7, 5, -1, 0, -1, -1],
             [9, 5, 7, 5, 9, 7, 8, -1, -1, -1, -1],
             [9, 5, 7, 5, 9, 7, 7, 0, -1, -1, -1],
             [9, 5, 7, 5, 9, 7, 9, -1, 0, -1, -1],
             [9, 5, 7, 7, 5, 9, 9, 0, -1, -1, -1],
             [9, 5, 7, 7, 5, 9, 7, -1, 0, -1, -1],
             [9, 5, 7, 7, 5, 9, 5, -1, -1, 0, -1],
             [9, 5, 7, 7, 5, 9, 7, -1, -1, -1, -1],
             [9, 5, 7, 7, 9, 5, 7, -1, 0, -1, -1],
             [9, 5, 7, 7, 9, 5, 7, 0, -1, -1, -1],
             [9, 5, 7, 7, 9, 5, 9, -1, -1, 0, -1],
             [9, 5, 7, 7, 9, 5, 7, -1, -1, 0, -1],
             [9, 5, 7, 9, 5, 7, 9, 0, -1, -1, -1],
             [9, 5, 7, 9, 5, 7, 9, -1, 0, -1, -1],
             [9, 5, 7, 9, 5, 7, 7, 0, -1, -1, -1],
             [9, 5, 7, 9, 5, 7, 7, -1, -1, 0, -1],
             [9, 5, 7, 9, 7, 5, 9, -1, -1, -1, -1],
             [9, 5, 7, 9, 7, 5, 7, -1, -1, -1, -1],
             [9, 5, 7, 9, 7, 5, 7, -1, -1, 0, -1],
             [9, 5, 7, 9, 7, 5, 5, -1, 0, -1, -1],
             [7, 6, 9, 6, 7, 9, 22, -1, -1, 0, -1],
             [7, 6, 9, 6, 7, 9, 22, 0, -1, -1, -1],
             [7, 6, 9, 6, 7, 9, 22, -1, 0, -1, -1],
             [7, 6, 9, 6, 7, 9, 22, -1, -1, -1, -1],
             [7, 6, 9, 6, 9, 7, 9, 0, -1, -1, -1],
             [7, 6, 9, 6, 9, 7, 24, -1, -1, -1, -1],
             [7, 6, 9, 6, 9, 7, 7, -1, -1, -1, -1],
             [7, 6, 9, 6, 9, 7, 7, 0, -1, -1, -1],
             [7, 6, 9, 7, 6, 9, 9, -1, 0, -1, -1],
             [7, 6, 9, 7, 6, 9, 6, 0, -1, -1, -1],
             [7, 6, 9, 7, 6, 9, 7, -1, -1, -1, -1],
             [7, 6, 9, 7, 6, 9, 22, -1, 0, -1, -1],
             [7, 6, 9, 7, 9, 6, 22, -1, -1, 0, -1],
             [7, 6, 9, 7, 9, 6, 9, -1, -1, -1, -1],
             [7, 6, 9, 7, 9, 6, 7, 0, -1, -1, -1],
             [7, 6, 9, 7, 9, 6, 6, -1, -1, -1, -1],
             [7, 6, 9, 9, 6, 7, 9, -1, -1, -1, -1],
             [7, 6, 9, 9, 6, 7, 22, -1, 0, -1, -1],
             [7, 6, 9, 9, 6, 7, 22, 0, -1, -1, -1],
             [7, 6, 9, 9, 6, 7, 22, -1, -1, -1, -1],
             [7, 6, 9, 9, 7, 6, 9, 0, -1, -1, -1],
             [7, 6, 9, 9, 7, 6, 9, -1, 0, -1, -1],
             [7, 6, 9, 9, 7, 6, 7, -1, -1, -1, -1],
             [7, 6, 9, 9, 7, 6, 22, -1, -1, -1, -1]])

y_np = np.array([[5],
             [6],
             [7],
             [5],
             [3],
             [5],
             [3],
             [3],
             [3],
             [3],
             [5],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [3],
             [3],
             [3],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [3],
             [3],
             [3],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [7],
             [1],
             [1],
             [7],
             [1],
             [4],
             [7],
             [2],
             [7],
             [7],
             [2],
             [1],
             [4],
             [2],
             [2],
             [7],
             [7],
             [7],
             [7],
             [7],
             [4],
             [1],
             [1],
             [4],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [7],
             [1],
             [1],
             [7],
             [4],
             [1],
             [4],
             [2],
             [7],
             [7],
             [7],
             [7],
             [4],
             [4],
             [1],
             [4],
             [7],
             [7],
             [2],
             [2],
             [4],
             [4],
             [7],
             [4],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [3],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [5],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [6],
             [7],
             [7],
             [2],
             [1],
             [1],
             [1],
             [4],
             [2],
             [3],
             [2],
             [4],
             [2],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [2],
             [1],
             [1],
             [1],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [7],
             [4],
             [2],
             [4],
             [4],
             [4],
             [1],
             [4],
             [4],
             [4],
             [4],
             [4],
             [2],
             [4],
             [4],
             [1],
             [1],
             [4],
             [4],
             [2],
             [2],
             [1],
             [1],
             [1],
             [4],
             [4],
             [2],
             [4],
             [4],
             [4],
             [4],
             [1],
             [4],
             [1],
             [4],
             [2],
             [4],
             [4],
             [1],
             [4],
             [4],
             [1],
             [4],
             [1],
             [2],
             [4],
             [1],
             [2],
             [1],
             [7],
             [7],
             [7],
             [7],
             [1],
             [7],
             [1],
             [1],
             [1],
             [2],
             [4],
             [7],
             [7],
             [1],
             [1],
             [1],
             [1],
             [7],
             [7],
             [7],
             [1],
             [1],
             [1],
             [7]])

In [215]:
# N is batch size; D_in is input dimension;
# H_{i} is the dimension of the ith hidden layer; D_out is output dimension.

N, D_in, H_1, H_2, H_3, D_out = 1102, 11, 128, 256, 256, 7

In [216]:
def multi_class_format(numpy_array):
    new = []
    for item in y_np:
        for i in item:
            new.append(i)
    
    class_labels = []
    
    for i in new:
        identifier = i-1
        class_labels.append(identifier)
        
    return class_labels

Y_array = multi_class_format(y_np)
print Y_array[:25]

[4, 5, 6, 4, 2, 4, 2, 2, 2, 2, 4, 2, 2, 2, 4, 4, 4, 2, 2, 2, 6, 6, 6, 6, 6]


In [217]:
nb_classes = 7
def get_one_hot(targets, nb_classes):
    return np.eye(nb_classes)[np.array(targets).reshape(-1)]

In [218]:
y_one_hot = get_one_hot(Y_array, 7)
print "Output shape: {} ".format(y_one_hot.shape)
print y_one_hot

Output shape: (1102, 7) 
[[ 0.  0.  0. ...,  1.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]
 ..., 
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]


In [219]:
# Make a train/test split using 20% test size
X_train, X_test, y_train, y_test = train_test_split(X_np, y_one_hot, test_size=0.20)
print "X_train: {}, y_train: {} , X_test: {} , y_test: {} ".format(X_train.shape[0], y_train.shape[0], X_test.shape[0], y_test.shape[0])

X_train: 881, y_train: 881 , X_test: 221 , y_test: 221 


In [220]:
X_train = torch.from_numpy(X_train).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)

X_test = torch.from_numpy(X_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

print "Training set shape: {} ".format(X_train.size())
print "Training labels shape: {} ".format(y_train.size())

print "Test set shape: {} ".format(X_test.size())
print "Test labels shape: {} ".format(y_test.size())

Training set shape: torch.Size([881, 11]) 
Training labels shape: torch.Size([881, 7]) 
Test set shape: torch.Size([221, 11]) 
Test labels shape: torch.Size([221, 7]) 


In [206]:
"""class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(11, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 7)
        
    def forward(self, X):
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X_out = self.fc3(X)
        return X_out"""

"""net = torch.nn.Sequential(
          torch.nn.Linear(D_in, H_1),
          torch.nn.ReLU(),
          torch.nn.Linear(H_1, H_2),
          torch.nn.ReLU(),
          torch.nn.Linear(H_2, D_out)
        )"""

net = torch.nn.Sequential(
    torch.nn.Linear(D_in, H_1),
    torch.nn.ReLU(),
    torch.nn.Linear(H_1, H_2),
    torch.nn.ReLU(),
    torch.nn.Linear(H_2, H_3),
    torch.nn.ReLU(),
    torch.nn.Linear(H_3, D_out)
    )

"""net = torch.nn.Sequential(
    torch.nn.Linear(D_in, D_out)
    )"""

'net = torch.nn.Sequential(\n    torch.nn.Linear(D_in, D_out)\n    )'

In [207]:
#net = Net()
print net

Sequential(
  (0): Linear(in_features=11, out_features=128)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=256)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=256)
  (5): ReLU()
  (6): Linear(in_features=256, out_features=7)
)


In [208]:
#Converting the training one hot vector labels into indices because PyTorch does not take one-hot vectors
targetnp = y_train.numpy()
idxs = np.where(targetnp > 0)[1]
new_targets = torch.LongTensor(idxs)

In [209]:
#Converting the test one hot vector labels into indices
test_targetnp = y_test.numpy()
test_idxs = np.where(test_targetnp > 0)[1]
new_test_targets = torch.LongTensor(test_idxs)

In [210]:
# create a Adam optimizer
learning_rate = 1e-3
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, weight_decay = 1e-2)
# create a loss function
loss_func = nn.CrossEntropyLoss()

In [211]:
#data = Variable(X, requires_grad = False)
#target = Variable(new_targets, requires_grad = False)

In [212]:
# run the main training loop
epochs = 2000
train_loss = 0
train_correct = 0
test_correct = 0
for epoch in range(epochs):
        data, target = Variable(X_train), Variable(new_targets)
        optimizer.zero_grad()
        net_out = net(data)
        #print net_out
        #print target.data
        
        #sum up the loss
        train_loss = loss_func(net_out, target)
        pred = net_out.data.max(1)[1] #get the index of the max log probabitlity
        train_correct = pred.eq(target.data).sum()
        
        train_loss.backward()
        optimizer.step()
        if epoch % 100 == 0:
            print('\nTraining Epoch: {} Average loss: {:.4f}, Training Accuracy: {}/{} ({:.4f}%)\n'.format(
        epoch, train_loss.data[0], train_correct, len(target),
        100. * train_correct / len(target)))



Training Epoch: 0 Average loss: 1.9595, Training Accuracy: 70/881 (7.9455%)


Training Epoch: 100 Average loss: 0.1751, Training Accuracy: 823/881 (93.4166%)


Training Epoch: 200 Average loss: 0.1351, Training Accuracy: 834/881 (94.6652%)


Training Epoch: 300 Average loss: 0.1186, Training Accuracy: 845/881 (95.9137%)


Training Epoch: 400 Average loss: 0.1184, Training Accuracy: 846/881 (96.0272%)


Training Epoch: 500 Average loss: 0.1159, Training Accuracy: 848/881 (96.2543%)


Training Epoch: 600 Average loss: 0.1135, Training Accuracy: 848/881 (96.2543%)


Training Epoch: 700 Average loss: 0.1107, Training Accuracy: 852/881 (96.7083%)


Training Epoch: 800 Average loss: 0.1092, Training Accuracy: 852/881 (96.7083%)


Training Epoch: 900 Average loss: 0.1077, Training Accuracy: 854/881 (96.9353%)


Training Epoch: 1000 Average loss: 0.1065, Training Accuracy: 853/881 (96.8218%)


Training Epoch: 1100 Average loss: 0.1055, Training Accuracy: 854/881 (96.9353%)


Training Epoch: 1

In [213]:
test_loss = 0
test_correct = 0

#Converting test data and target labels into pyTorch variables
test_data, test_target = Variable(X_test), Variable(new_test_targets)

#Test set output
net_out = net(test_data)

test_loss = loss_func(net_out, test_target)
test_pred = net_out.data.max(1)[1] #get the index of max log probability
test_correct = test_pred.eq(test_target.data).sum()

print('\nTest loss: {:.4f}, Test accuracy: {}/{} ({:.4f}%)\n'.format(
    test_loss.data[0], test_correct, len(test_target), 100. * test_correct / len(test_target)))


Test loss: 0.1124, Test accuracy: 212/221 (95.9276%)

